In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

# test stereo aruco

In [2]:
from pkg.marker_config import *

In [3]:
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect

In [4]:
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.83100015, -0.03025655, -0.5554488 ,  0.64721805],
        [ 0.17196472,  0.96358234,  0.20478582, -0.14104328],
        [ 0.5290245 , -0.26569465,  0.8059401 ,  0.07000864],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

In [5]:
def get_T_rel(coord_from, coord_to, objectPose_dict):
    return np.matmul(SE3_inv(objectPose_dict[coord_from]), objectPose_dict[coord_to])

def detect_robots(detector, robot_names, ref_name='floor'):
    xyz_rpy_robots = {}
    while True:
        try:
            objectPose_dict = detector.detect(name_mask=robot_names+[ref_name])

            for rname in robot_names:
                Tbr = get_T_rel(ref_name, rname, objectPose_dict)
                xyz_rpy_robots[rname] = T2xyzrpy(Tbr)

            break
        except KeyError as e:
            print(e)
            break
        except Exception as e:
            print(e)
            pass
    T0 = np.identity(4)
    return xyz_rpy_robots

In [6]:
xyz_rpy_robots = detect_robots(stereo, robot_names=['indy0', 'panda1'])

# init robots

In [7]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ([-0.44648051261901855, 0.251528263092041, 0.009795188903808594],
                                     [7.722511439072125e-05, 0.012837732857963772, -1.5843305292051728]),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ([0.5117020606994629, 0.16830319166183472, 0.014661192893981934],
                                     [0.0037190383704881766, 0.013066991871646852, -1.6051065831214242]),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


In [13]:
__rospy_initialized, __roscore

NameError: name '__rospy_initialized' is not defined

In [12]:
ROBOTS_ON_SCENE, xyz_rpy_robots, custom_limits = crob.robots_on_scene, xyz_rpy_robots, crob.custom_limits
node_name='task_planner'
start_rviz=True
custom_xacro=None

if not __rospy_initialized:
    __roscore = subprocess.Popen(['roscore'])
    rospy.init_node(node_name, anonymous=True)
    __rospy_initialized = True

urdf_content = None
xcustom = XacroCustomizer.instance()
if custom_xacro is not None:
    xcustom.initialize(ROBOTS_ON_SCENE, xyz_rpy_robots, xacro_path=custom_xacro)
else:
    xcustom.initialize(ROBOTS_ON_SCENE, xyz_rpy_robots)

JOINT_NAMES, LINK_NAMES, urdf_content = \
    xcustom.convert_xacro_to_urdf(
        joint_fix_dict={'finger': 'upper'},
        joint_offset_dict={},
        joint_limit_dict=custom_limits)

if start_rviz:
    xcustom.start_rviz()

xcustom, joint_names, link_names, urdf_content = xcustom, JOINT_NAMES, LINK_NAMES, urdf_content

NameError: name '__rospy_initialized' is not defined

In [8]:
xcustom, joint_names, link_names, urdf_content = set_custom_robots(crob.robots_on_scene, xyz_rpy_robots, crob.custom_limits, start_rviz=True)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


# init geometry handle

In [9]:
ghnd = GeometryHandle(urdf_content, joint_names, link_names, rviz=True)

Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
object_pose_dict =  stereo.detect()

In [11]:
Tref = object_pose_dict['floor']
for oname, opose in object_pose_dict.items():
    atem = aruco_map[oname]
    if atem.dlevel == DetectionLevel.ROBOT:
        continue
    xyzrpy = T2xyzrpy(np.matmul(SE3_inv(Tref),opose))
    ghnd.create_safe(gtype=atem.gtype, name=atem.name, dims=atem.dims, center=xyzrpy[0], rpy=xyzrpy[1], link_name="base_link",
                     color=atem.color, fixed=DetectionLevel.is_fixed(atem.dlevel), display=True, collision=True
                    )

Please create a subscriber to the marker


# Disconnect stereo

In [16]:
stereo.disconnnect()